# **Лабораторная работа №3**
## Ссылка на гит - https://github.com/igor2271814/Nikolashin_Python_Laba_1

In [2]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet


# --- 1. Загрузка CSV без заголовков ---
file_path = "Laptop_price.csv"
try:
    df = pd.read_csv(file_path, header=None)
    print("[+] CSV файл успешно загружен.")
except Exception as e:
    print(f"[!] Ошибка загрузки файла: {e}")
    exit()


# --- 2. Добавляем названия столбцов ---
column_names = ['Brand', 'CPU_Speed', 'RAM', 'Storage', 'Screen_Size', 'Weight', 'Price']
df.columns = column_names
print("[+] Столбцам присвоены заголовки:", column_names)


# --- 3. Проверка на CSV-Injection ---
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].astype(str).str.startswith(dangerous_chars).any():
            print(f"[!] Найдены потенциальные CSV-инъекции в столбце '{col}'!")
        else:
            print(f"[+] Столбец '{col}' безопасен.")


check_csv_injection(df)


# --- 4. Фильтрация от SQL/XSS инъекций ---
def clean_input(value):
    val = str(value)

    # SQL ключевые слова
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION"]
    for keyword in sql_keywords:
        if keyword.lower() in val.lower():
            return "[BLOCKED]"

    # SQL комментарии
    sql_comments = ["--", "/*", "*/"]
    for comment in sql_comments:
        if comment in val:
            return "[BLOCKED]"

    # XSS паттерны
    xss_patterns = [
        r"<script.*?>.*?</script>",
        r"javascript:.*",
        r"onerror=.*"
    ]
    for pattern in xss_patterns:
        if re.search(pattern, val, re.IGNORECASE):
            return "[BLOCKED]"

    return value


# Применяем фильтр ко всем строкам
df = df.applymap(lambda x: clean_input(x))
print("[+] Очистка данных от SQL/XSS инъекций завершена.")


# --- 5. Хеширование цены (SHA-256) ---
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()


df['Price_Hashed'] = df['Price'].apply(hash_price)
print("[+] Цена захеширована и добавлена в столбец 'Price_Hashed'.")


# --- 6. Шифрование данных (Fernet AES) ---
key = Fernet.generate_key()
cipher = Fernet(key)


def encrypt_data(data):
    return cipher.encrypt(str(data).encode()).decode()


def decrypt_data(encrypted_data):
    return cipher.decrypt(encrypted_data.encode()).decode()


# Шифруем цену
df['Price_Encrypted'] = df['Price'].apply(encrypt_data)
print("[+] Цена зашифрована и добавлена в столбец 'Price_Encrypted'.")

# Шифруем RAM
df['RAM_Encrypted'] = df['RAM'].apply(encrypt_data)
print("[+] RAM зашифрован и добавлен в столбец 'RAM_Encrypted'.")

# Расшифровываем первые 5 значений RAM
decrypted_ram_samples = [decrypt_data(val) for val in df['RAM_Encrypted'].head(5)]
print("[+] Пример расшифрованных значений RAM:", decrypted_ram_samples)


# --- 7. Сохранение защищённого CSV ---
output_path = "Laptop_price_secured.csv"
df.to_csv(output_path, index=False)
print(f"[+] Обработанные данные сохранены в файл: {output_path}")

[+] CSV файл успешно загружен.
[+] Столбцам присвоены заголовки: ['Brand', 'CPU_Speed', 'RAM', 'Storage', 'Screen_Size', 'Weight', 'Price']
[+] Столбец 'Brand' безопасен.
[+] Столбец 'CPU_Speed' безопасен.
[+] Столбец 'RAM' безопасен.
[+] Столбец 'Storage' безопасен.
[+] Столбец 'Screen_Size' безопасен.
[+] Столбец 'Weight' безопасен.
[+] Столбец 'Price' безопасен.
[+] Очистка данных от SQL/XSS инъекций завершена.
[+] Цена захеширована и добавлена в столбец 'Price_Hashed'.
[+] Цена зашифрована и добавлена в столбец 'Price_Encrypted'.
[+] RAM зашифрован и добавлен в столбец 'RAM_Encrypted'.
[+] Пример расшифрованных значений RAM: ['RAM_Size', '16', '4', '4', '16']
[+] Обработанные данные сохранены в файл: Laptop_price_secured.csv


C:\Users\User\AppData\Local\Temp\ipykernel_24024\431576129.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_input(x))
